In [1]:
import time
import requests
import json
import os
from bs4 import BeautifulSoup
from datetime import datetime
import pandas as pd
from selenium import webdriver
import datetime
from multiprocessing.pool import ThreadPool

In [ ]:
os.chdir('C:\Users\dmolitor\Dropbox\Fordham\GA Support\Research\Summer 2018\Experiment_COM\Data')

In [7]:
def get_all_projects():
    project_urls = []
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64; Trident/7.0; Touch; rv:11.0) like Gecko',
    }
    req = requests.get('https://experiment.com/discover/more?offset=0')
    total_project_count = int(str(json.loads(req.content)['count']).strip())
    #total_project_count = 12 
    for i in range(0, total_project_count):
        req = requests.get('https://experiment.com/discover/more?offset=' + str(i), headers=headers)
        content = json.loads(req.content)
        soup = BeautifulSoup(content['cards'], 'html.parser')

        for project in soup.find_all('div', {'class': 'project-card-content'}):
            project_url = project.find('a', {'class': 'plain project-link home-page:clicked-project-card'}).get('href')
            project_urls.append('https://experiment.com' + project_url)
    return list(set(project_urls))

def init_driver(url=None):
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument('--no-sandbox')

        
    CHROMEDRIVER_PATH = r"C:\\Users\dmolitor\\Downloads\\chromedriver_win32\\chromedriver.exe"  
    driver = webdriver.Chrome(CHROMEDRIVER_PATH, options=options)
    driver.get(url)
    time.sleep(2)
    return driver

def crawl_projects(testing=False):
    all_project_results = []
    if testing:
        project_urls = [
            "https://experiment.com/projects/do-zoonotic-pathogens-exist-in-fijian-bats-and-if-so-what-are-the-risks-of-exposure-to-humans",
            "https://experiment.com/projects/exploring-the-bioremediation-and-environmental-impact-of-halogenated-organic-compounds"
        ]
    else:
        project_urls = get_all_projects()
    for each_project_url in project_urls:
        try:
            driver = init_driver(url=each_project_url)

            title = driver.find_element_by_xpath('//h1[@class="title"]').text

            researchers = [a.text
                # {a.find_element_by_xpath('.//a').get_attribute('href'): a.text}
                           for a in driver.find_elements_by_xpath('//*[@id="team"]//div[@class="researcher"]//div[@class="full-name"]')]
            lab_notes = [
                {a.find_element_by_tag_name('a').get_attribute('href'): a.find_element_by_xpath(
                    './/div[@class="title"]').text}
                for a in driver.find_elements_by_xpath('//div[@class="labnote-card"]')]
            backers = driver.find_element_by_xpath('//ul[@class="backer-stats"]//span[@class="stat-number"]').text
            tags = [a.text for a in driver.find_elements_by_xpath('//div[@class="tags"]/a')]
            category = [a.text for a in driver.find_elements_by_xpath('//a[contains(@class, "tag category")]')] # tag category
            try:
                about = driver.find_element_by_xpath('//section[@id="about"]//p').text
            except:
                about = driver.find_element_by_xpath('//section[@id="about"]').text
            try:
                budgets = driver.find_element_by_xpath('(//div[@class="funding-raised"]//span)[3]').text
            except:
                budgets = None

            endorsed_by = [a.text for a in
                           driver.find_elements_by_xpath('//div[@class="endorsement"]//div[@class="name"]')]
            time.sleep(0.1)
            try:
                show_more_btn = driver.find_element_by_xpath('//div[@class="show-more"]/a').click()
            except:
                pass

            timeline = [a for a in
                        driver.find_elements_by_xpath('//div[@class="milestones-timeline"]//div[@class="description"]')]
            timeline_events = []
            for t in timeline:
                try:

                    timeline_events.append(
                        [t.find_element_by_tag_name("h4").text, t.find_element_by_tag_name("h2").text])

                except:
                    pass

            try:
                beg_timeline = datetime.strptime(timeline_events[0][0], '%b %d, %Y')
                end_timeline = datetime.strptime(timeline_events[-1][0], '%b %d, %Y')

                days = (end_timeline - beg_timeline).days
            except:
                days = None
                pass
            project_fund_percentage = driver.find_element_by_xpath(
                '//li[@class="backer-stat second"]//span[@class="stat-number"]').text
            project_fund_rased = driver.find_element_by_xpath(
                '//div[@class="funding-raised"]//span[@class="focus-stat"]').text


            try:
                number_of_images = len([driver.find_elements_by_xpath('//div[@class="hero-image"]')])
            except:
                number_of_images = None
                print("No Image:::", each_project_url)
            try:
                number_of_videos = len([driver.find_elements_by_xpath('//div[@class="hero-image js-video"]')])
            except:
                number_of_videos = None
                print("No Video:::", each_project_url)

            try:
                goal = driver.find_element_by_xpath('(//div[@class="funding-bar-stats"]//div[@class="stat"]//span)[2]').text#still funding
            except:
                goal = driver.find_element_by_xpath('(//div[@class="description text-antialiased"])[1]').text

            try:
                time_left = driver.find_element_by_xpath('//div[@class="stat float-right text-right"]//span').text #still funding
            except:
                time_left = driver.find_element_by_xpath('(//div[@class="description text-antialiased"])[2]').text#finish funding


            all_project_results.append({"title": title,
                                        'author': researchers[0],
                                        "researchers": '; '.join(researchers),
                                        "category": '; '.join(category),
                                        "number_of_researchers": len(researchers),
                                        "backers": backers,
                                        "tags": tags,
                                        "number_of_tags": len(tags),
                                        "about": about,
                                        "budgets": budgets,
                                        "number_of_budget_segments": len(budgets) if budgets else None,
                                        "project_fund_percentage": project_fund_percentage,
                                        "project_fund_rased": project_fund_rased,
                                        "number_of_images": number_of_images,
                                        "number_of_videos": number_of_videos,
                                        "days": days,
                                        "timeline_events": timeline_events,
                                        "number_of_timeline_events": len(timeline_events) if timeline_events else None,
                                        "lab_notes": lab_notes,
                                        "number_of_lab_notes": len(lab_notes) if lab_notes else None,
                                        "endorsed_by": '; '.join(endorsed_by),
                                        "number_of_endorsed_by": len(endorsed_by) if endorsed_by else None,
                                        'url':each_project_url,
                                        'time_left':time_left,
                                        'goal':goal.replace('Raised of'and 'Goal','')
                                        })

            driver.close()
            print("Project Detail Success:::", title)
        except Exception as e:
            print("Project Detail Crawl Error:::", each_project_url, str(e))
            pass

    df = pd.DataFrame(all_project_results)
    filePath = 'All_Projects_' + str(datetime.datetime.today().strftime('%m%d')) + '.csv'
    df.to_csv(filePath, encoding='utf8')
    print('Done')

In [8]:
def test():
    crawl_projects(testing=False)
#     df = pd.read_pickle('project.pkl')
#     df.to_csv('All_projects_Wk_11_16_18.csv',encoding='utf8')

('Project Detail Success:::', u'Spiders of Yasuni: How do their Physical Traits Determine their Lifestyle?')
('Project Detail Success:::', u"Mothers' feelings and their children's activity levels")
('Project Detail Success:::', u'The Dogs of Chernobyl Research Initiative')
('Project Detail Success:::', u'Azolla, a little fern with massive green potential')
('Project Detail Success:::', u'Rarest cat in the world? Assessing the Status of the Javan Fishing Cat')
('Project Detail Success:::', u'Personalized Cancer and Viral Therapy: Clostridium-based Cell Delivery System coupled to CRISPR/Cas9 Nanotherapeutic')
('Project Detail Success:::', u'Why do wolverines need snow?')
('Project Detail Success:::', u'Unraveling the mystery of TDP-43, a versatile protein involved in several Neurodegenerative disorders')
('Project Detail Success:::', u'Are coyotes contributing to the spread Rocky Mountain spotted fever in Arizona?')
('Project Detail Success:::', u'Is the Felixer cat trap safe for native 

('Project Detail Success:::', u'Genetics of gypsum-endemism and drought tolerance in Sporobolus grass')
('Project Detail Success:::', u'Sustainable Living on Mars: Remediation of Martian Soil to Produce Oxygen')
('Project Detail Success:::', u'How does Mount Rainier help maintain traditional tribal plant harvesting?')
('Project Detail Success:::', u'Tracking Magellanic Penguins')
('Project Detail Success:::', u'Living Skin Therapeutics')
('Project Detail Success:::', u'Freshwater Biodiversity Crisis: Is Restoration the Key?')
('Project Detail Crawl Error:::', u'https://experiment.com/projects/join-bighorn-basin-paleontological-institute-s-2018-field-expedition', 'Message: no such element: Unable to locate element: {"method":"xpath","selector":"//section[@id="about"]"}\n  (Session info: headless chrome=70.0.3538.102)\n  (Driver info: chromedriver=2.43.600210 (68dcf5eebde37173d4027fa8635e332711d2874a),platform=Windows NT 6.1.7601 SP1 x86_64)\n')
('Project Detail Success:::', u"Early-Stag

('Project Detail Success:::', u'Can Data Management Tools Improve Research Efficiency and Reproducibility?')
('Project Detail Success:::', u'How do Bobtail Squid choose their glowing bacterial partner?')
('Project Detail Success:::', u'An 8000-year History of Climate and Environmental Change in P\xe1ramo of Central Costa Rica')
('Project Detail Success:::', u'Bring a Triceratops to Seattle')
('Project Detail Success:::', u'The Mystery of the Disappearing Seabirds: Using Science to Protect Caribbean Seabirds')
('Project Detail Success:::', u'Developing A New Treatment For Neurodegenerative Diseases')
('Project Detail Success:::', u'Learning to Live, or Living to Learn?')
('Project Detail Success:::', u'Penguins, plants, and people: Getting to the core of climate change in the Falkland Islands')
('Project Detail Success:::', u"Obtaining clean, renewable energy from 'dead zones'")
('Project Detail Success:::', u'Birds eavesdrop on plant indirect defenses to locate insect preys')
('Project

('Project Detail Success:::', u'Trapping to Uncover the Secrets of the California Moray Eel, a Kelp Forest Predator')
('Project Detail Success:::', u'Environmental Justice in North Charleston Communities')
('Project Detail Success:::', u'Record Voices of The Sizang Language of Burma')
('Project Detail Success:::', u'Targeted Drug Delivery by using Magnetic Nanoparticles')
('Project Detail Success:::', u'Refining and improving techniques to increase survival of macaw chicks using foster macaw parents in the wild')
('Project Detail Success:::', u'MeadoWatch: When do Mt. Rainier Wildflowers Bloom?')
('Project Detail Success:::', u'Reducing scientific research waste production: Can we recycle lab plastics for 3D printing?')
('Project Detail Success:::', u'Can a Meal Replacement Help Solve Hunger in America?')
('Project Detail Success:::', u'Non-target effect of insecticide seed treatment on milkweeds and monarch butterflies')
('Project Detail Success:::', u'Do humans and noise pollution im

('Project Detail Success:::', u'Digging up an Exceptionally Large Dinosaur Graveyard')
('Project Detail Success:::', u'Did the Pima Indians design arrow points specifically to penetrate Apache rawhide shields?')
('Project Detail Success:::', u'Monitoring and Banding Birds in the Sacramento Valley of California')
('Project Detail Success:::', u'Assessing Guadalupe Fur Seal Health and Ecology')
('Project Detail Success:::', u'What Animals Live in Disturbed Sumatran Rainforest? A Camera Trap Study')
('Project Detail Success:::', u'Cannibalism in Giant Tyrannosaurs')
('Project Detail Success:::', u'UWashington Formula Motorsports: Electric Racecar Development')
('Project Detail Success:::', u'Improving the detection of expired medication in developing countries to improve maternal health')
('Project Detail Success:::', u'The Effects of High Flow Rates on the Giant Mayfly Nymphs and Their Burrows')
('Project Detail Success:::', u'Mommy dearest: how maternal care shapes juvenile stress in rh

...

('Project Detail Success:::', u'New bioactive peptides from skin of a colombian frog')
('Project Detail Success:::', u'Can hair loss be reversed with Oxy133, a sterol based drug candidate?')
('Project Detail Success:::', u'How do birds see the world?')
('Project Detail Success:::', u'Capturing canine personality')
('Project Detail Success:::', u'Placenta-on-a-Chip: Ethical and Universal Drug Testing Using Microfluidics')
('Project Detail Success:::', u'ZomBees: Are parasites controlling the minds of honey bees?')
('Project Detail Success:::', u'Assessing oyster restoration using morphological records and next-generation sequencing')
('Project Detail Success:::', u'Lemur Boot Camp! Challenging survival skills for reintroduction')
('Project Detail Success:::', u'i-cows: can intimidating eye patterns painted onto cows reduce lion attacks?')
('Project Detail Success:::', u'Do Electronic Cigarettes Promote Smoking Cessation?')
('Project Detail Success:::', u'Monitoring carbon sequestration 

('Project Detail Success:::', u'Development of non-invasive diagnostics for endotheliotropic herpesvirus in free-ranging Asian elephants')
('Project Detail Success:::', u'Climate change threats to the American pika')
('Project Detail Success:::', u'View into Vocalizations, making affordable tags from smartwatches to track humpback whales and what they are saying')
('Project Detail Success:::', u'A conservation comic book: Involving Rwandan children in saving endangered Grey Crowned Cranes')
('Project Detail Success:::', u'Comprehensive Conservation of Southern Resident Killer Whales in the Modern Ocean')
('Project Detail Success:::', u'Designing a fish identification survey app for use underwater')
('Project Detail Success:::', u'What kind of microbes attach to floating plastic in coastal waters?')
('Project Detail Success:::', u'Dino damage and death assemblages: Analyzing ailments and environments of Morrison sauropods')
('Project Detail Success:::', u'Reputation Formation in Domesti

('Project Detail Success:::', u'Can a crocodile cure your ills? Testing the Crocodylus Apothecary Hypothesis')
('Project Detail Success:::', u'Using aphids to measure electrical outputs in ferns')
('Project Detail Success:::', u'How do amphibious fishes find their way around on land?')
('Project Detail Success:::', u'Through a glass darkly: assessing population size of an endangered cave salamander from samples of spring and cave water')
('Project Detail Success:::', u'Evaluating women\u2019s experiences with a referral system for safe and legal abortion care on the Thailand-Burma border')
('Project Detail Success:::', u'What can we do to prevent depression from coming back again?')
('Project Detail Success:::', u'Satellite Tracking Reef Manta Rays in Papua New Guinea to Inform Conservation Management')
('Project Detail Success:::', u'The role of local communities in influencing environmental policies; Case Study: Madre de Dios, Peru')
('Project Detail Success:::', u'Squid in Space: Sy

('Project Detail Success:::', u'How do mixed species social groups help fairy-wrens?')
('Project Detail Success:::', u'Application of Graphene into an Electric Vehicle')
('Project Detail Success:::', u'Battle of the clocks: Does your biological clock determine whether the timing of exercise impairs or promotes sleep?')
('Project Detail Success:::', u'Identifying the genes necessary to regenerate an injured brain - insights from a basal chordate, Ciona intestinalis')
('Project Detail Success:::', u'Filling the Knowledge Gap: Seismic Hazards at Cherry Point, WA')
('Project Detail Success:::', u'How does hemlock tree decline impact bird bioindicators and ecosystem condition?')
('Project Detail Success:::', u'Surviving on the savanna: how do traits predict tree success?')
('Project Detail Success:::', u'Clean Indoor Air with Plants in Specially Engineered Pots')
('Project Detail Success:::', u'MyHealthEd')
('Project Detail Success:::', u"Karst Springs Initiative: Measuring Tennessee's Larg

('Project Detail Success:::', u'Looking for Lost Sharks: An Exploration of Discovery through the Western Indian Ocean')
('Project Detail Success:::', u'Making Moorit: Determining the Molecular Basis for Brown Colored Sheep')
('Project Detail Success:::', u'Developing a modular paper-based detection device for Tuberculosis')
('Project Detail Success:::', u'Identifying the best habitats for the Caribbean manatee in Cuba using sonar')
('Project Detail Success:::', u'Exploring Molecules and Microbes')
('Project Detail Success:::', u'Part II: Continuing linguistic research and developing materials for literacy education in Salasaka Kichwa')
('Project Detail Success:::', u'Striped bass diet, health, and movement patterns: Using science to inform management')
('Project Detail Success:::', u'Does group gender composition influence how we explain success or failure?')
('Project Detail Success:::', u'Lemur loving: are the mate choices of the critically endangered blue-eyed black lemur affecting 

In [11]:
schedule.every().day.at("11:42").do(test)